<a href="https://colab.research.google.com/github/pinilDissanayaka/DocRAG/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install langchain
!pip install --upgrade --quiet unstructured
!pip install langchain_community
!pip uninstall sentence-transformers
!pip install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cach

In [22]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA

In [12]:
urls = [
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023",
    "https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023",
]

loader=UnstructuredURLLoader(urls=urls)
documents=loader.load()

In [13]:
documents[1]

Document(metadata={'source': 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-9-2023'}, page_content='Skip to main content\n\nSearch form\n\nHome\n\nWho We Are\n\nResearch\n\nPublications\n\nGet Involved\n\nPlanned Giving\n\nDonate\n\nRussian Offensive Campaign Assessment, February 9, 2023\n\nFeb 9, 2023 - ISW Press\n\nDownload the PDF\n\nKarolina Hird, Riley Bailey, George Barros, Nicole Wolkov, and Frederick W. Kagan\n\nFebruary 9, 7:30 pm ET\n\nClick here to see ISW’s interactive map of the Russian invasion of Ukraine. This map is updated daily alongside the static maps present in this report.\n\nWagner Group financier Yevgeny Prigozhin announced on February 9 that the Wagner Group has entirely stopped recruiting prisoners.\xa0In a response to a press comment, Prigozhin claimed that Wagner’s recruitment of prisoners has "completely stopped" and that "all obligations are being fulfilled" for those currently under Wagner’s employ.[1]\xa0Pri

In [18]:
textSplitter=RecursiveCharacterTextSplitter(separators=['\n', '\n \n', ' '], chunk_size=500, chunk_overlap=100)
splitedDocuments=textSplitter.split_documents(documents)

In [19]:
splitedDocuments[2]

Document(metadata={'source': 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-february-8-2023'}, page_content='the Kharkiv-Luhansk Oblast border, particularly northwest of Svatove near Kupyansk and west of Kreminna.[1]\xa0Geolocated combat footage has confirmed Russian gains in the Dvorichne area northwest of Svatove.[2]\xa0Russian military command additionally appears to have fully committed elements of several conventional divisions to decisive offensive operations along the Svatove-Kreminna line, as ISW previously reported.[3]\xa0Elements of several regiments of the 144th\xa0and 3rd\xa0Motor Rifle Division')

In [24]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
vectoreStore=FAISS.from_documents(documents=splitedDocuments, embedding=embeddings)